# Coroutines et tâches
Cette séance donne un aperçu des API de haut-niveau du module asyncio pour utiliser les coroutines et les tâches.

## Coroutines
Les coroutines déclarées avec la syntaxe async/await sont la manière privilégiée d’écrire des applications asyncio. Par exemple, l’extrait de code suivant (requiert Python 3.7+) affiche « hello », attend 1 seconde et affiche ensuite « world » :

In [5]:
import nest_asyncio
import asyncio

In [4]:
import asyncio

async def main():
     print('hello')
     await asyncio.sleep(1)
     print('world')

asyncio.run(main())

hello
world
